# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings

warnings.filterwarnings('ignore')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file



llm_model = "gpt-4.1-nano"


In [9]:
import pandas as pd
df = pd.read_csv('assets/Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [75]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [72]:
model = ChatOpenAI(
    model=llm_model,
    temperature=0.7
)

In [17]:
# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "What is the best company name for a company that makes {product}?"
)
# Create a chain
chain = prompt | model | StrOutputParser()
# Test the chain
result = chain.invoke({"product": "colorful socks"})
print(result)

Here are some creative and catchy company name ideas for a colorful sock brand:

1. **Sock Spectrum**  
2. **ColorPop Socks**  
3. **Vivid Toes**  
4. **Rainbow Steps**  
5. **BrightStride**  
6. **Chromatic Socks**  
7. **Hue & Sole**  
8. **ColorSplash Socks**  
9. **KaleidoSocks**  
10. **Prism Socks**  

When choosing the best name, consider factors like uniqueness, availability of domain names, and how well it reflects your brand’s personality.


### Sequential Chains (Single Input → Single Output)

In [18]:
name_prompt = ChatPromptTemplate.from_template(
    "Suggest a company name for a business that makes {product}."
)

name_chain = name_prompt | model | StrOutputParser()

In [19]:
description_prompt = ChatPromptTemplate.from_template(
    "Write a 20-word description for a company called {company_name}."
)

description_chain = description_prompt | model | StrOutputParser()

In [20]:
from langchain_core.runnables import RunnableLambda

overall_chain = (
    name_chain
    | RunnableLambda(lambda name: {"company_name": name})
    | description_chain
)

In [21]:
result = overall_chain.invoke({"product": "queen-size sheet set"})
print(result)

Sure! Sure! offers luxurious, regal queen-size sheet sets designed for ultimate comfort and elegant sleep experiences.


### 3️⃣ Multi-Input / Multi-Output Chains

In [22]:
translate_prompt = ChatPromptTemplate.from_template(
    "Translate this review to English:\n\n{review}"
)

translate_chain = translate_prompt | model | StrOutputParser()

In [23]:
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize this review in one sentence:\n\n{english_review}"
)

summary_chain = summary_prompt | model | StrOutputParser()

In [24]:
language_prompt = ChatPromptTemplate.from_template(
    "What language is this review written in?\n\n{review}"
)

language_chain = language_prompt | model | StrOutputParser()

Run multiple chains in parallel

In [ ]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

initial_step = RunnableParallel(
    english_review=translate_chain,
    language=language_chain,
)

Add summary step

In [31]:
full_chain = (
    initial_step
    | RunnableParallel(
        english_review=itemgetter("english_review"),
        language=itemgetter("language"),
        summary=summary_chain
    )
)
result = full_chain.invoke({"review": df.Review[5]})
print(result)

{'english_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones from stores and the flavor is much better... Old batch or counterfeit!?", 'language': 'The review is written in French.', 'summary': 'The reviewer finds the taste mediocre and the foam unsatisfactory, suspecting it may be an old or counterfeit batch compared to store-bought versions.'}


In [30]:
followup_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up response to this summary in {language}:\n\n{summary}"
)

followup_chain = followup_prompt | model | StrOutputParser()

final_chain = full_chain | followup_chain
result = final_chain.invoke({"review": df.Review[5]})
print(result)

Je vous remercie pour votre retour. Il est regrettable que la saveur et la mousse n’aient pas été à la hauteur de vos attentes. Nous prenons ces remarques très au sérieux et allons immédiatement vérifier la qualité de nos fournisseurs pour nous assurer de la fraîcheur et de l’authenticité de nos produits. Nous espérons pouvoir améliorer notre offre et vous satisfaire lors de votre prochaine visite.


## Router Chain

In [58]:
physics_chain = (
    ChatPromptTemplate.from_template(
        "PHYSICS CHAIN\nYou are a physics expert. Answer this question:\n{question}"
    )
    | model
    | StrOutputParser()
)

math_chain = (
    ChatPromptTemplate.from_template(
        "MATH CHAIN\nYou are a math expert. Answer this question:\n{question}"
    )
    | model
    | StrOutputParser()
)

general_chain = (
    ChatPromptTemplate.from_template(
        "GENERAL CHAIN\nYou are a 2 year old childAnswer this question:\n{question}"
    )
    | model
    | StrOutputParser()
)

In [ ]:
from langchain_core.runnables import RunnableBranch

# simple routing using string matching
simple_router = RunnableBranch(
    (lambda x: "physics" in x["question"].lower(), physics_chain),
    (lambda x: "integral" in x["question"].lower(), math_chain),
    general_chain
)

In [ ]:
simple_router.invoke({"question": "What is blackbody radiation?"})

'Hmm, blackbody radiation is when things get really hot and they give off light all by themselves, like the sun or a hot stove!'

#### Routing using a classifer for the question

In [90]:
# Implement a classifier that can classify the question into one of the following categories:
# - physics
# - math
# - general
# The classifier should return the category name.

from langchain_core.runnables import RunnableLambda, RunnablePassthrough

classifier_prompt = ChatPromptTemplate.from_messages([
    ("system", "Classify the question into exactly one category: physics, math, or general. Reply with only that single word."),
    ("human", "{question}"),
])
classifier_chain = classifier_prompt | model | StrOutputParser()


def _normalize(s):
    raw = (s or "").strip()
    category = raw.lower().split()[0] if raw else "general"
    print(f"🚀 [classifier] selected category: {category}")
    return category


# Single Runnable: input -> input with "category" set (physics|math|general).
# Run this before RunnableBranch so conditions read category without re-invoking the LLM.
classifier_runnable = (
    RunnablePassthrough.assign(
        category=classifier_chain | RunnableLambda(_normalize)
    )
)


# Router: classifier_runnable runs once; RunnableBranch conditions use x["category"]
classifier_router = classifier_runnable | RunnableBranch(
    (lambda x: x.get("category") == "physics", physics_chain),
    (lambda x: x.get("category") == "math", math_chain),
    general_chain,
)


In [89]:
result = classifier_router.invoke({"question": "What is blackbody radiation?"})
print(result)

Blackbody radiation refers to the electromagnetic radiation emitted by an idealized object known as a blackbody, which absorbs all incident radiation regardless of frequency or angle. When a blackbody is heated to a certain temperature, it emits a characteristic spectrum of radiation that depends solely on its temperature, not on its material composition. 

The spectrum of blackbody radiation has a distinct shape: it peaks at a wavelength inversely proportional to the temperature (according to Wien's displacement law) and its intensity distribution follows the Planck radiation law. This concept was crucial in resolving the ultraviolet catastrophe predicted by classical physics and led to the development of quantum theory, as Planck introduced quantization of energy to accurately describe the observed spectrum. 

In essence, blackbody radiation provides a foundational understanding of how objects emit electromagnetic energy due to their temperature, serving as a cornerstone in thermodyn

In [85]:
classifier_router.invoke({"question": "What is photosynthesis"})

🚀 [classifier] selected category: general


'Photosynthesis is when plants use sunlight to make their food. They take in sunlight, water, and air, and turn it into something they can eat to grow.'

In [84]:
classifier_router.invoke({"question": "What is polygamy"})

🚀 [classifier] selected category: general


'Polygamy is when someone has more than one husband or wife at the same time.'

In [87]:
result = classifier_router.invoke({"question": "Explain probability theory"})
print(result)

🚀 [classifier] selected category: math
Probability theory is a branch of mathematics that deals with analyzing and modeling uncertainty and randomness. It provides a formal framework for quantifying the likelihood of different events occurring.

**Key Concepts in Probability Theory:**

1. **Sample Space (Ω):** The set of all possible outcomes of a random experiment. For example, when rolling a die, the sample space is {1, 2, 3, 4, 5, 6}.

2. **Events:** Subsets of the sample space. An event could be "rolling an even number," which corresponds to {2, 4, 6}.

3. **Probability Measure (P):** A function that assigns a number between 0 and 1 to each event, representing the likelihood of that event. The probability of the entire sample space is 1, indicating certainty.

4. **Basic Axioms of Probability:**
   - Non-negativity: \( P(A) \geq 0 \) for any event \( A \).
   - Normalization: \( P(Ω) = 1 \).
   - Additivity: For mutually exclusive events \( A \) and \( B \), \( P(A \cup B) = P(A) +